In [ ]:
!pip install farasapy

In [1]:
import requests
import json
from bs4 import BeautifulSoup
import re

In [ ]:
url = "https://alarabi.nccal.gov.kw/Home/Article/"
articles = []
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
segmentor = arabic_segmentor()
for i in range(1000, 2000):
  response = requests.get(url+str(i), headers= headers)
  if response.status_code==200:
    print(i, end=" ")
    html = BeautifulSoup(response.content)
    details = html.find("ul", {"class": "post-info-dark"})
    author = details.find_all("a")[0].text.strip()
    section = details.find_all("a")[1].text.strip()
    issue = details.find_all("a")[2].text.strip()
    issue = re.sub(r'[^0-9]', '', issue)
    paragraphs = [x.text for x in html.find("div", {"class": "news-details-layout1"}).find("div", {"class": "row"}).find_all("p")]
    paragraphs = [re.sub('["\':;\(\)-_$»«/]', '', p) for p in paragraphs]
    paragraphs = [re.sub('\.', ' . ', p) for p in paragraphs]
    paragraphs = [re.sub(',', ' ، ', p) for p in paragraphs]
    paragraphs = [re.sub('؟', ' ؟ ', p) for p in paragraphs]
    paragraphs = [re.sub('!', ' ! ', p) for p in paragraphs]
    paragraphs = [re.sub('،', ' ، ', p) for p in paragraphs]
    paragraphs = [paragraph for paragraph in paragraphs if paragraph != ' ']
    text = " ".join([re.sub('\s+', ' ', p) for p in paragraphs])
    text = re.sub('\s+', ' ', text)
    text = segmentor.segment(text)
    text = re.sub('\+ة', 'ة', text)
    text = re.sub('\+ا', 'ا', text)
    articles += [{"author": author, "section": section, "issue": issue, "text": text}]

In [ ]:
json.dump(articles, open("alaraby1k.json", "w"))

In [223]:
import json, re

def detokenizer(word, symbol):
    """ Takes segmented arabic words using 'symbol' and returns the word
    tokenized by classifying prefixs and suffixes
    """
    if not symbol in word: return word
    prefixes = ["ال", "ل", "و", "ب", "ف", "س", "لال", "وال", "بال"]
    suffixes = ["ت", "ه" , "ها", "ات", "ين", "هم", "ي", "ون", "وا", "نا", "هما"]
    common = ["ك"]
    parts = word.split(symbol)
    tokens = []
    for i, part in enumerate(parts):
        if part in prefixes:
            tokens.append(f"{part}+")
        elif part in suffixes:
            tokens.append(f"+{part}")
        elif part in common:
            if i==0:
                tokens.append(f"{part}+")
            else:
               tokens.append(f"+{part}") 
        else:
            tokens.append(f"{part}")
    return " ".join(tokens)

In [231]:
alaraby1k = json.load(open("alaraby1k.json", "r"))
for i, article in enumerate(alaraby1k):
    text = article["text"]
    words = text.split()
    words = [ detokenizer(w, "+") for w in words]
    text = " ".join(words)
    article["text"] = text
json.dump(alaraby1k, open("alaraby1k.json", "w"))